### Basic import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pretrainedModel import *
from faceRecognition import *
import pandas as pd
from math import *
from matplotlib import pyplot as plt

### FaceNet pre-trained model

In [ ]:
sess, embeddings, images_in, phase_train_in, pnet, rnet, onet = detectionNetwork()

### Set up face DB

In [ ]:
# Path of the references faces that will construct the database of faces
pathsKnown = ["/home/sabrine/Pictures/laurene1.jpg",
              "/home/sabrine/Pictures/laurene2.jpg",
              "/home/sabrine/Pictures/laurene3.jpg",
              "/home/sabrine/Pictures/wissem1.jpg",
              "/home/sabrine/Pictures/wissem2.jpg",
              "/home/sabrine/Pictures/wissem3.jpg",
              "/home/sabrine/Pictures/sabrineb1.jpg",
              "/home/sabrine/Pictures/sabrineb2.jpg",
              "/home/sabrine/Pictures/sabrineb3.jpg",
             "/home/sabrine/Pictures/oliverb1.jpg",
             "/home/sabrine/Pictures/oliver2.jpg",
             "/home/sabrine/Pictures/oliverb3.jpg",
             "/home/sabrine/Pictures/lucasb1.jpg",
             "/home/sabrine/Pictures/lucasb2.jpg",
             "/home/sabrine/Pictures/lucas3.jpg",
             "/home/sabrine/Pictures/albertob1.jpg",
             "/home/sabrine/Pictures/albertob2.jpg",
             "/home/sabrine/Pictures/albertob3.jpg"]

# Creation of identities names
names = ['Laurene', 'Laurene', 'Laurene', 'Wissem', 'Wissem', 'Wissem','Sabrine', 'Sabrine', 'Sabrine', 'Oliver',
         'Oliver', 'Oliver', 'Lucas', 'Lucas', 'Lucas', 'Alberto', 'Alberto', 'Alberto']

height= [1.8, 1.8, 1.8, 1.83, 1.83, 1.83, 1.44125278245, 1.44125278245, 1.44125278245, 1.57004028797,
        1.57004028797, 1.57004028797, 1.46793014233, 1.46793014233, 1.46793014233, 1.47711563941, 1.47711563941,
         1.47711563941]

In [ ]:
# detection parameters
minsize = 40 
threshold = [ 0.75, 0.75, 0.75 ]
factor = 0.709

In [ ]:
df = embeddingKnownFaces(pathsKnown, names, height, sess, embeddings, images_in,
                   phase_train_in, pnet, rnet, onet,
                   minsize, threshold, factor)

In [ ]:
pd.to_pickle(df, 'database')

### Retreive rgbSeq with detection information

In [ ]:
synchronize = pd.read_pickle('synchronization/synchronizeS')

In [ ]:
# Generate a list of name in the same order of the known embeddings without repetition 
uniqueNames = []
for i in names:
    if i not in uniqueNames:
        uniqueNames.append(i)
print (uniqueNames)

In [ ]:
'''
FR for all frames - store returned information in the same order of the df synchronize
'''

bigSmallestArray = []
bigCenters = []
#scores = [[] for i in range(len(uniqueNames))]
embs = []
rgb = []
faceX = []
faceY = []
blurScore = []
#frontalBool = []

for seq in synchronize.rgbSeq.unique():
    #Path of the frames
    fileNumber = str("%03d"%seq)
    path = '/home/sabrine/notebook/framesRGB/' + fileNumber + '.jpg'
    
    # List of names, coordinates and distances of the FR process
    fr = faceRecognition(pathsKnown, names, path, sess, embeddings, images_in, phase_train_in, pnet, rnet, onet, df)
    emb = fr[0]
    centers = fr[1] 
    blur = fr[2]

    # check if at least one face has been detected
    if len(fr[0]) > 0:
        for i in range(len(blur)):
            bigCenters.append(centers[i])
            faceX.append(centers[i][0])
            faceY.append(centers[i][1])
            embs.append(emb[i])
            rgb.append(seq)
            blurScore.append(blur[i])
    else:
        # So the index of rgb, bigCenters and scores will be the same as synchronize.rgbSeq.unique()
        # It will simplify the join process of these 2 df
        rgb.append(np.nan)
        faceX.append(np.nan)
        faceY.append(np.nan)
        embs.append(np.nan)
        blurScore.append(np.nan)
        bigCenters.append(np.nan)

#### Check if all list has the same size 

In [ ]:
len(embs)

In [ ]:
len(faceY)

In [ ]:
len(faceX)

In [ ]:
len(bigCenters)

In [ ]:
len(rgb)

In [ ]:
len(blurScore)

#### Creation of the df faces

In [ ]:
faces = pd.DataFrame(np.asarray([faceX, faceY, rgb, blurScore]).T, 
                     columns=['faceX', 'faceY', 'rgb', 'blurScore'])
faces['embs'] = embs
faces.dropna(inplace=True)

In [ ]:
faces.head()

In [ ]:
# save the result in pickle algorithm
faces.to_pickle('facesS')
np.save('uniqueName', uniqueNames)